## Phase 4: Model Evaluation

Now we'll evaluate the model's performance on the test set and visualize the results.

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_test_reshaped = y_test.reshape(-1, 1)
y_pred_reshaped = y_pred.reshape(-1, 1)

# Create dummy array with same shape as input data
dummy = np.zeros((len(y_test_reshaped), X_train.shape[2]))
dummy[:, 0] = y_test_reshaped.flatten()
y_test_inv = scaler.inverse_transform(dummy)[:, 0]

dummy[:, 0] = y_pred_reshaped.flatten()
y_pred_inv = scaler.inverse_transform(dummy)[:, 0]

# Calculate metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

print(f'Mean Absolute Error: ${mae:.2f}')
print(f'Root Mean Squared Error: ${rmse:.2f}')

In [ ]:
def plot_predictions(y_true, y_pred, start_idx=0, n_hours=72):
    plt.figure(figsize=(15, 6))
    
    # Plot actual vs predicted prices
    plt.plot(y_true[start_idx:start_idx+n_hours], label='Actual', marker='o')
    plt.plot(y_pred[start_idx:start_idx+n_hours], label='Predicted', marker='o')
    
    plt.title('Actual vs Predicted Electricity Prices')
    plt.xlabel('Hours')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot predictions for a sample period
plot_predictions(y_test_inv, y_pred_inv)

# Plot prediction error distribution
plt.figure(figsize=(10, 6))
errors = y_test_inv - y_pred_inv
sns.histplot(errors, kde=True)
plt.title('Prediction Error Distribution')
plt.xlabel('Error ($)')
plt.ylabel('Count')
plt.show()

## Phase 5: Model Deployment

Now we'll create a function to make predictions for the next 24 hours using the most recent data.

In [ ]:
def predict_next_24_hours(model, recent_data, scaler):
    # Ensure recent_data is scaled
    scaled_data = scaler.transform(recent_data)
    
    # Reshape for prediction
    X = scaled_data[-24:].reshape(1, 24, recent_data.shape[1])
    
    # Make prediction
    pred_scaled = model.predict(X)
    
    # Inverse transform prediction
    dummy = np.zeros((24, recent_data.shape[1]))
    dummy[:, 0] = pred_scaled.flatten()
    predictions = scaler.inverse_transform(dummy)[:, 0]
    
    return predictions

# Get the most recent data
recent_features = df_processed[features].values[-24:]

# Make predictions
next_24_hours = predict_next_24_hours(model, recent_features, scaler)

# Plot predictions
plt.figure(figsize=(12, 6))
plt.plot(range(24), next_24_hours, marker='o')
plt.title('24-Hour Price Forecast')
plt.xlabel('Hour')
plt.ylabel('Predicted Price ($)')
plt.grid(True)
plt.show()

# Print hourly predictions
print('
Hourly Price Predictions for Next 24 Hours:')
for hour, price in enumerate(next_24_hours):
    print(f'Hour {hour:2d}: ${price:.2f}')